In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import cross_val_score
sns.set()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_audi = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/audi.csv')
data_audi['brand'] = 'Audi'
data_bmw = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/bmw.csv')
data_bmw['brand'] = 'BMW'
data_ford = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/ford.csv')
data_ford['brand'] = 'Ford'
data_hyundi=pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/hyundi.csv')
data_hyundi['brand']='Hyundi'
data_hyundi = data_hyundi.rename(columns={'tax(£)': 'tax'})
data_merc =pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/merc.csv')
data_merc['brand'] = 'Merc'
data_skoda = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/skoda.csv')
data_skoda['brand'] = 'Skoda'
data_toyota = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/toyota.csv')
data_toyota['brand']= 'Toyota'
data_vauxhall = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/vauxhall.csv')
data_vauxhall['brand'] = 'Vauxhall'
data_vw = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/vw.csv')
data_vw['brand'] = 'VW'

### Merging and preprocessing

In [ ]:
data = pd.concat([data_audi, data_bmw, data_ford, data_hyundi,
                  data_merc, data_skoda, data_toyota, data_vauxhall, data_vw], ignore_index=True)

In [ ]:
data

In [ ]:
data.describe(include='all')

In [ ]:
data.info()

In [ ]:
# reducing the size of the data set

data["model"] = data["model"].astype("category")
data["transmission"] = data["transmission"].astype("category")
data["fuelType"] = data["fuelType"].astype("category")
data["brand"] = data["brand"].astype("category")

# Downcasting
data[["year", "price", "mileage", "tax", "mpg"]] = data[["year", "price", "mileage", "tax", "mpg"]].apply(pd.to_numeric, downcast="unsigned")

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

## correlation scores with Heatmap

In [ ]:
corr = data.corr()

sns.heatmap(corr, annot=True)

### 3.1) Plotted the graphs with numerical features vs price


In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(2,5,1)
plt.scatter(data['year'],data['price'])
plt.title('price and year')
plt.subplot(2,5,2)
plt.scatter(data['engineSize'],data['price'])
plt.title('price and engineSize')
plt.subplot(2,5,3)
plt.scatter(data['mileage'],data['price'])
plt.title('price and mileage')
plt.subplot(2,5,4)
plt.scatter(data['mpg'],data['price'])
plt.title('price and mpg')
plt.subplot(2,5,5)
plt.scatter(data['tax'],data['price'])
plt.title('price and tax')


plt.show()

In [ ]:
sns.distplot(data['price'])

Price has big tail at the end of the distribution so its better to apply log scale for price

In [ ]:
data['log_price'] = np.log(data['price']) 

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(2,5,1)
plt.scatter(data['year'],data['log_price'])
plt.title('log_price and year')
plt.subplot(2,5,2)
plt.scatter(data['engineSize'],data['log_price'])
plt.title('log_price and engineSize')
plt.subplot(2,5,3)
plt.scatter(data['mileage'],data['log_price'])
plt.title('log_price and mileage')
plt.subplot(2,5,4)
plt.scatter(data['mpg'],data['log_price'])
plt.title('log_price and mpg')
plt.subplot(2,5,5)
plt.scatter(data['tax'],data['log_price'])
plt.title('log_price and tax')


plt.show()

After log transformation of price its looks linear in most of the graphs

In [ ]:
plt.figure(figsize=(12,8))
data.drop(['price'], axis = 1).boxplot()

#### As per the box plot its important to do scaling

## Scaling the inputs(features)

In [ ]:
# scalling
from sklearn.preprocessing import StandardScaler
features = ['year', 'mileage', 'mpg', 'engineSize', 'tax']
dummies = ['transmission', 'fuelType', 'brand']
X = data[features]
data_scaler = StandardScaler()

data_scaler.fit(X)

data_trans=data_scaler.transform(X)

data_scaled = pd.DataFrame(data_trans, columns=[features])
data_scaled

In [ ]:
data_transform = pd.concat([data_scaled,data[dummies]], axis =1)
data_transform

In [ ]:
cols = ['year','mileage','mpg','engineSize','tax','transmission','fuelType', 'brand']

data_transform.columns = cols
data_transform.head()

In [ ]:
plt.figure(figsize=(12,8))
data_scaled.boxplot()

## Create the dummie variables

In [ ]:
data_dummies = pd.get_dummies(data_transform, drop_first= True)

data_dummies

### 3.4)Declaring the X and y variables 


In [ ]:
X = data_dummies
y= data['log_price']

### Testing the Linear regression models (OLS, Lasso, Ridge)


In [ ]:
def test_models(models, X,y, iterations = 100):
    results = {}
    for i in models:
        r2_train = []
        r2_test = []
        for j in range(iterations):
            x_train, x_test, y_train, y_test = train_test_split(X,y,test_size= 0.2)
            models[i].fit(x_train,y_train).predict(x_train)
            r2_train.append(models[i].fit(x_train,y_train).score(x_train, y_train))
            models[i].fit(x_train,y_train).predict(x_test)
            r2_test.append(models[i].fit(x_train,y_train).score(x_test, y_test))
            
            results[i] = [np.mean(r2_train), np.mean(r2_test)]
           
    return pd.DataFrame(results)

In [ ]:
models = {'OLS': LinearRegression(),'Lasso': Lasso(),'Ridge': Ridge()}

test_models(models, X, y)

### Train test split

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)

### 6.2) GradientBoostingRegressor model

In [ ]:
boosting = GradientBoostingRegressor(n_estimators= 100, learning_rate= 0.1, max_depth=10)
boosting.fit(x_train, y_train)
y_pred_train = boosting.predict(x_train)
y_pred_test =boosting.predict(x_test)

In [ ]:
mse_gb_train= mean_squared_error(np.exp(y_train), np.exp(y_pred_train))
mse_gb_test= mean_squared_error(np.exp(y_test), np.exp(y_pred_test))
mse_abs_gb_train = mean_absolute_error(np.exp(y_train), np.exp(y_pred_train))
mse_abs_gb_test = mean_absolute_error(np.exp(y_test), np.exp(y_pred_test))                                    

rmse_gb_train = np.sqrt(mse_gb_train)
rmse_gb_test = np.sqrt(mse_gb_test)

print("Train:", mse_gb_train)
print("Test:", mse_gb_test)
print("Train:", mse_abs_gb_train)
print("Test:", mse_abs_gb_test)
print("Train:", rmse_gb_train)
print("Test:", rmse_gb_test)

In [ ]:
r2_boosting_test = boosting.score(x_test,y_test )
r2_boosting_train = boosting.score(x_train,y_train )
print(r2_boosting_train)
r2_boosting_test

In [ ]:
scores_boosting = pd.DataFrame(data = np.round([r2_boosting_train, r2_boosting_test, mse_gb_train, mse_gb_test, rmse_gb_train, rmse_gb_test, mse_abs_gb_train, mse_abs_gb_test],2), index =['r2_train', 'r2_test', 'mse_train', 'mse_test', 'rmse_train', 'rmse_test', 'mse_abs_train', 'mse_abs_test'], columns=['scores'])
scores_boosting

In [ ]:
score_b= cross_val_score(boosting, X, y, cv=20, n_jobs=-1)
score_b

### RandomForestRegressor Model

In [ ]:
random_forest_reg = RandomForestRegressor(max_depth = 20, max_features = 'auto', n_estimators = 100)
random_forest_reg.fit(x_train, y_train)

y_pred_train = random_forest_reg.predict(x_train)
y_pred_test = random_forest_reg.predict(x_test)

In [ ]:
r2_rfr_test = random_forest_reg.score(x_test, y_test)
r2_rfr_train = random_forest_reg.score(x_train, y_train)
r2_rfr_test, r2_rfr_train

In [ ]:
mse_rfg_train= mean_squared_error(np.exp(y_train), np.exp(y_pred_train))
mse_rfg_test= mean_squared_error(np.exp(y_test), np.exp(y_pred_test))
mse_abs_rfg_train = mean_absolute_error(np.exp(y_train), np.exp(y_pred_train))
mse_abs_rfg_test = mean_absolute_error(np.exp(y_test), np.exp(y_pred_test))                                    

rmse_rfg_train = np.sqrt(mse_rfg_train)
rmse_rfg_test = np.sqrt(mse_rfg_test)

print("Train:", mse_rfg_train)
print("Test:", mse_rfg_test)
print("Train:", mse_abs_rfg_train)
print("Test:", mse_abs_rfg_test)
print("Train:", rmse_rfg_train)
print("Test:", rmse_rfg_test)

In [ ]:
scores_randomforest = pd.DataFrame(data = np.round([r2_rfr_train, r2_rfr_test, mse_rfg_train, mse_rfg_test, rmse_rfg_train, rmse_rfg_test, mse_abs_rfg_train, mse_abs_rfg_test],2), index =['r2_train', 'r2_test', 'mse_train', 'mse_test', 'rmse_train', 'rmse_test', 'mse_abs_train', 'mse_abs_test'], columns=['scores'])
scores_randomforest

### Decision Trees for Regression

In [ ]:
tree_regressor = DecisionTreeRegressor(max_depth=15)
tree_regressor.fit(x_train, y_train)
y_pred_train = tree_regressor.predict(x_train)
y_pred_test = tree_regressor.predict(x_test)

In [ ]:
mse_dtr_train= mean_squared_error(np.exp(y_train), np.exp(y_pred_train))
mse_dtr_test= mean_squared_error(np.exp(y_test), np.exp(y_pred_test))
mse_abs_dtr_train = mean_absolute_error(np.exp(y_train), np.exp(y_pred_train))
mse_abs_dtr_test = mean_absolute_error(np.exp(y_test), np.exp(y_pred_test))                                    

rmse_dtr_train = np.sqrt(mse_dtr_train)
rmse_dtr_test = np.sqrt(mse_dtr_test)

print("Train:", mse_dtr_train)
print("Test:", mse_dtr_test)
print("Train:", mse_abs_dtr_train)
print("Test:", mse_abs_dtr_test)
print("Train:", rmse_dtr_train)
print("Test:", rmse_dtr_test)

In [ ]:
r2_dt_test = tree_regressor.score(x_test,y_test )
r2_dt_train = tree_regressor.score(x_train,y_train )
print(r2_dt_train)
r2_dt_test

In [ ]:
scores_DT = pd.DataFrame(data = np.round([r2_dt_train, r2_dt_test, mse_dtr_train, mse_dtr_test, rmse_dtr_train, rmse_dtr_test, mse_abs_dtr_train, mse_abs_dtr_test], 2), index =['r2_train', 'r2_test', 'mse_train', 'mse_test', 'rmse_train', 'rmse_test', 'mse_abs_train', 'mse_abs_test'], columns=['scores'])
scores_DT

### Overview of models

In [ ]:
models_with_scores= pd.concat([scores_DT, scores_boosting, scores_randomforest], axis =1)

cols = ['DecisionTrees', 'GradiantBoosting', 'RandomForest']

models_with_scores.columns = cols


In [ ]:
models_with_scores

From the above results I choose RandomForest model for future predictions. 